In [1]:
import pandas as pd
import numpy as np
from lenskit import batch,topn
from lenskit.metrics import topn as tn
from lenskit.algorithms import als
from lenskit.matrix import  CSR,RatingMatrix
from scipy import spatial
import matplotlib.pyplot as plt
from lenskit import topn
from IPython.core.debugger import set_trace
from lenskit.metrics import predict


# Data Prep

In [465]:
train=pd.read_csv('/project/naray190/ml-20m/ratings.csv')
test=pd.read_csv('/project/naray190/ml-20m/truncated_user_ratings.csv')
#train=train[~train['user'].isin(np.unique(test['user']))]
#test=train.sample(frac=0.2)
#train=train.drop(test.index)

In [466]:
train=train[['userId','movieId','rating']]
test=test[['userId','movieId','rating']]
train.columns = ['user','item','rating']
test.columns=['user','item','rating']


In [464]:
len(train)

103745

In [463]:
len(test)

4185

## Rating Normalization

In [6]:
gbias=train['rating'].mean()
train['rating']-=gbias
test['rating']-=gbias

In [7]:

group=train.groupby('item')['rating']
item_biases=group.sum()/(group.count()+5)
train=train.join(pd.DataFrame(item_biases),on="item",how="inner",rsuffix="_im")
train=train.assign(rating=lambda df:df.rating-df.rating_im)
test=test.join(pd.DataFrame(item_biases),on="item",how="inner",rsuffix="_im")
test=test.assign(rating=lambda df:df.rating-df.rating_im)

In [8]:
group=train.groupby('user')['rating']
user_biases_train=group.sum()/(group.count()+5)
train=train.join(pd.DataFrame(user_biases_train),on="user",how="inner",rsuffix="_um")
train=train.assign(rating=lambda df:df.rating-df.rating_um)
group=test.groupby('user')['rating']
user_biases_test=group.sum()/(group.count()+5)
test=test.join(pd.DataFrame(user_biases_test),on="user",how="inner",rsuffix="_um")
test=test.assign(rating=lambda df:df.rating-df.rating_um)


In [9]:
train=train[['user','item','rating']]
test=test[['user','item','rating']]

# Folding-In Function

In [93]:
class FoldIn(als.BiasedMF):
    def __init__(self,*args,**kwargs):
        super (FoldIn,self).__init__(*args,**kwargs)
        self.bias=None
    def fold_in(self,new_ratings):
        #set_trace()
        rmat, users, items = sparse_ratings(new_ratings,iidx=self.item_index_)
        n_users = len(users)
        n_items = len(items)
        
        
        umat = np.full((n_users, self.features), np.nan)
        #set_trace()
        umat = als._train_matrix(rmat.N, self.item_features_, self.regularization)
        #set_trace()

        return umat,users
    

def sparse_ratings(ratings, scipy=False,uidx=None,iidx=None):
    """
    Convert a rating table to a sparse matrix of ratings.
    Args:
        ratings(pandas.DataFrame): a data table of (user, item, rating) triples.
        scipy: if ``True``, return a SciPy matrix instead of :py:class:`CSR`.
    Returns:
        RatingMatrix:
            a named tuple containing the sparse matrix, user index, and item index.
    """
    #set_trace()
    if(uidx is None): 
        uidx = pd.Index(ratings.user.unique(), name='user')
    if(iidx is None):
        iidx = pd.Index(ratings.item.unique(), name='item')
    

    row_ind = uidx.get_indexer(ratings.user).astype(np.int32)
    col_ind = iidx.get_indexer(ratings.item).astype(np.int32)

    if 'rating' in ratings.columns:
        vals = np.require(ratings.rating.values, np.float64)
    else:
        vals = None

    matrix = CSR.from_coo(row_ind, col_ind, vals, (len(uidx), len(iidx)))
    #set_trace()
    if scipy:
        matrix = CSR.to_scipy(matrix)

    return RatingMatrix(matrix, uidx, iidx)

algo=FoldIn(features=25,iterations=50,reg=0.1)
algo.fit(train)
regularumat=algo.user_features_
poponlyumat,popuindex=algo.fold_in(test)


# User Similarity and Baseline Pairwise Similarity

In [95]:
testusers=test['user'].unique().tolist()
user_simscore=pd.DataFrame(columns=['user','simscore'])
r=0

for user in testusers:
    indexf = algo.user_index_.get_loc(user)
    indexp= popuindex.get_loc(user)
    full_v=regularumat[indexf]
    pop_v=poponlyumat[indexp]
    dot_product=1-spatial.distance.cosine(full_v,pop_v)
    user_simscore.loc[r]=[user,dot_product]
    r=r+1

In [96]:
pairwise_sim_score=0
count=0
for user1 in testusers:
    
    if testusers.index(user1) != len(testusers)-1:
        user2=testusers[testusers.index(user1)+1]
        ind1=algo.user_index_.get_loc(user1)
        ind2=algo.user_index_.get_loc(user2)
        u1v=regularumat[ind1]
        u2v=regularumat[ind2]
        dot_product=1-spatial.distance.cosine(u1v,u2v)
        pairwise_sim_score+=dot_product
        count+=1
    


In [ ]:
pairwise_sim_score/count

In [98]:
pairwise_sim_score_pop=0
count=0
for user1 in testusers:
    
    if testusers.index(user1) != len(testusers)-1:
        user2=testusers[testusers.index(user1)+1]
        ind1=popuindex.get_loc(user1)
        ind2=popuindex.get_loc(user2)
        u1v=poponlyumat[ind1]
        u2v=poponlyumat[ind2]
        dot_product=1-spatial.distance.cosine(u1v,u2v)
        pairwise_sim_score_pop+=dot_product
        count+=1
    


In [99]:
pairwise_sim_score_pop/count

0.0010522642093688651

In [22]:
users=test.user.unique().tolist()

In [75]:
itemmat=algo.item_features_

In [119]:
predicts=pd.DataFrame(columns=['user','item','score','rank'])

In [124]:
pop_predicts=pd.DataFrame(columns=['user','item','score','rank'])

# Scoring the items here

In [504]:
#I'm scoring the items here
def recommend_for_users(algo,users,user_index,user_matrix,predicts,train,k):
    for user in users:
        uix=user_index.get_loc(user)
        uvfull=user_matrix[uix]
        user_movies=train.loc[train['user']==user]
        movie_list=set(user_movies['item'].tolist())
        candidates=set(train['item'].unique())-movie_list
        remove_movie=set(movie_data.loc[movie_data["popularity"]<10].item.values)
        candidates=candidates-remove_movie
        candidates=list(candidates)
        iix=algo.lookup_items(candidates)
    #for movie in candidates:
        #iix=algo.item_index_.get_loc(movie)
        score=np.matmul(algo.item_features_[iix],uvfull)
        score=score+item_biases.loc[candidates]+user_biases_train[user]+gbias
        scores=pd.DataFrame({"item":candidates,"score":score})
        scores['user']=user
    
    
        #if(np.isnan(score)):
        #    score=0
    
        #newrow=pd.DataFrame([[user,movie,score]],columns=['user','item','score'])
        #scores=scores.append(newrow)
    
   # z=sum(1*scores['score']==0)
  #  if(z==0):
  #      print("No zeroes for user ",user) 
    #print(np.min(scores['score']),np.max(scores['score']),user)
    
   # if(np.min(scores['score']))
    #score_df=pd.DataFrame(scores)
    #score_df['user']=user
    #score_df.reset_index(level=0,inplace=True)
    #score_df['item'] = score_df["item"].apply(lambda x: algo.item_index_[x])
    #score_df=score_df[['user','item','rating']]
    #score_df = score_df[~score_df['item'].isin(movie_list)]
    #score_df.columns=['user','item','score']
        scores=scores.sort_values('score',ascending=False)
        scores=scores.head(k)
        scores['rank']=scores['score'].rank(ascending=0)
        predicts=predicts.append(scores,sort=True)
    return predicts

NameError: name 'recommend_for_users' is not defined

In [125]:
for user in users:
    uix=popuindex.get_loc(user)
    uvfull=poponlyumat[uix]
    user_movies=train.loc[train['user']==user]
    movie_list=set(user_movies['item'].tolist())
    candidates=set(train['item'].unique())-movie_list
    remove_movie=set(movie_data.loc[movie_data["popularity"]<10].item.values)
    candidates=candidates-remove_movie
    candidates=list(candidates)
    
    iix=algo.lookup_items(candidates)
    #for movie in candidates:
        #iix=algo.item_index_.get_loc(movie)
    score=np.matmul(algo.item_features_[iix],uvfull)
    score=score+item_biases.loc[candidates]+user_biases_train[user]+gbias
    scores=pd.DataFrame({"item":candidates,"score":score})
    scores['user']=user
    
        #if(np.isnan(score)):
        #    score=0
    
        #newrow=pd.DataFrame([[user,movie,score]],columns=['user','item','score'])
        #scores=scores.append(newrow)
    
   # z=sum(1*scores['score']==0)
  #  if(z==0):
  #      print("No zeroes for user ",user)
    #print(np.min(scores['score']),np.max(scores['score']),user)
   
        
    
   # if(np.min(scores['score']))
    #score_df=pd.DataFrame(scores)
    #score_df['user']=user
    #score_df.reset_index(level=0,inplace=True)
    #score_df['item'] = score_df["item"].apply(lambda x: algo.item_index_[x])
    #score_df=score_df[['user','item','rating']]
    #score_df = score_df[~score_df['item'].isin(movie_list)]
    #score_df.columns=['user','item','score']
    scores=scores.sort_values('score',ascending=False)
    scores=scores.head(20)
    scores['rank']=scores['score'].rank(ascending=0)
    pop_predicts=pop_predicts.append(scores)


/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavio

/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavio

/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavio

/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavio

/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavio

...

/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavio

/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavio

/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavio

/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavio

/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/project/naray190/drproj/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavio

In [126]:
pop_predicts

,item,rank,score,user
100553,100553,1.0,4.771151,65
527,527,2.0,4.515141,65
858,858,3.0,4.498818,65
113315,113315,4.0,4.491111,65
50,50,5.0,4.468869,65
26587,26587,6.0,4.458498,65
1193,1193,7.0,4.449967,65
65357,65357,8.0,4.442095,65
6016,6016,9.0,4.438147,65
94466,94466,10.0,4.428370,65


,item,score,user
123571,123571,6.048877,128468
8619,8619,6.001222,128468


,item,popularity,avgRating
25491,123571,10,3.35


,user,count,popcount,unpopcount,percentpop
128467,128468,140,134,6,0.957143


In [167]:
train.loc[train.user == 128468 and (train.rating>0)]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [151]:
train.loc[train.item == 131062]

,user,item,rating
11528436,79570,131062,-2.879231


In [81]:
scores=scores.sort_values('score',ascending=True)
predicts

NameError: name 'predictslen' is not defined

In [172]:
algo.item_features_[algo.item_index_.get_loc(56915)]

array([-4.21612728,  4.61644663,  5.71737175,  0.34292515, -7.86367504,
        2.69034353, -0.25748642, -7.17450531,  2.6887351 , -2.16220624,
       -4.64642979,  4.95412621,  9.49599656,  2.64345712, -4.35644024,
        3.14399068, 11.3387658 ,  3.40636242, -0.59594358,  0.37594519,
        1.52165961, -5.41269138, -4.96951457, -5.87281762, -4.5820669 ,
        3.24226765,  7.807597  ,  6.42410159,  3.63964829,  1.92440816])

In [149]:
algo.item_features_[algo.item_index_.get_loc(1)]

array([-0.1189633 ,  0.16093052,  0.00213796,  0.0202751 , -0.05038016,
        0.09155937, -0.11458383, -0.0046035 ,  0.01272063,  0.04847082,
       -0.12204939,  0.04692655, -0.04743933, -0.05128105, -0.02109792,
        0.01144532, -0.07482805,  0.01053487, -0.00777533, -0.1281133 ,
        0.01180843,  0.00841388, -0.15293832, -0.03703078,  0.06183735,
       -0.03342402,  0.0176976 , -0.13225396, -0.00799472, -0.08131214])

In [130]:
algo.item_features_[algo.item_index_.get_loc(3281)]

array([ 13.96894122, -12.53720425, -10.53347939,   6.53447849,
       -10.06237497,   3.63612665,   6.13140088,   5.24157123,
         1.45842344,   6.50141854,  -5.62916788, -10.19704129,
         1.83486223,   0.70870048, -20.38485362,  -8.76102702,
        -4.75127386,   7.50417358,  13.44998069, -11.20246504,
         7.96626814,   3.9451022 ,  15.76197385,  -0.89963726,
        -3.421989  ,   4.40985691,   2.05988731,  18.77531855,
        12.03232996,   2.09085974])

In [150]:
algo.item_features_[algo.item_index_.get_loc(3281)]

array([-0.06204119,  0.11630173,  0.13446888,  0.19671371, -0.11930791,
        0.05030458,  0.11690685, -0.14446743,  0.13946569, -0.1889363 ,
        0.03762516,  0.19003015,  0.07467158,  0.40296395, -0.38578006,
       -0.11056076,  0.14069715,  0.17063566,  0.08004215,  0.10739705,
       -0.1325016 ,  0.11098331, -0.09432157, -0.11446602, -0.13404113,
        0.14347887, -0.1548189 , -0.04928973, -0.04870412,  0.11488589])

In [161]:
algo.user_features_[algo.user_index_.get_loc(42912)]

array([-0.02516115,  0.18396442, -0.16452006, -0.26243026,  0.22476   ,
        0.14832164, -0.06188056,  0.35177349,  0.07249231,  0.12012601,
        0.16723538, -0.20322359, -0.06928984, -0.10270993,  0.28603066,
       -0.18157056, -0.20590046, -0.29527595, -0.16815102,  0.27757442,
       -0.12214259,  0.29413032,  0.06784915,  0.14257029,  0.14711506,
       -0.30343723,  0.01805842,  0.04661552, -0.28932049, -0.05665237])

In [143]:
predicts.to_csv("ALS25FT20RecsReg01fwithrank.tsv",sep="\t",index=None)

In [144]:
pop_predicts.to_csv("ALS25FT20RecsPopProfileReg01fwithrank.tsv",sep="\t",index=None)

In [169]:
algo.predict_for_user(42912,[88670])

88670   -0.128297
dtype: float64

In [26]:
movie_data=pd.read_csv("/project/naray190/movie_data_20M.csv",sep=",")

In [30]:
movie_data.columns=['item','popularity','avgRating']

In [39]:
user_data=pd.read_csv("/project/naray190/user_data_20M.csv",sep=",")

In [41]:
user_data

,userId,count,popcount,unpopcount,percentpop
0,1,175,152,23,0.868571
1,2,61,48,13,0.786885
2,3,187,179,8,0.957219
3,4,28,28,0,1.000000
4,5,66,66,0,1.000000
5,6,24,24,0,1.000000
6,7,276,240,36,0.869565
7,8,70,70,0,1.000000
8,9,35,28,7,0.800000
9,10,38,38,0,1.000000


In [69]:
sketchy_predicts

,user,item,score


In [42]:
user_data.columns=['user','count','popcount','unpopcount','percentpop']

In [91]:
movie_impressions=pd.DataFrame(columns=['item','impression'])

In [92]:
movie=set()
for index, row in predicts.iterrows():
    
    movie_id=row['item']
    if(movie_id in movie):
        mirow=movie_impressions.loc[movie_impressions['item'] == movie_id]
        count=mirow['impression']+1
        movie_impressions.impression.loc[movie_impressions['item'] ==  movie_id] = count
    
    else:
        newrow=pd.DataFrame([[movie_id,1]], columns=['item','impression'])
        movie.add(movie_id)    
        movie_impressions=movie_impressions.append(newrow)
            

In [265]:
mov_data=pd.merge(movie_impressions,movie_data,on=['item'])

In [299]:
algo.predict_for_user(79570,[113315])+item_biases.loc[113315]+user_biases_train[79570]+gbias

113315    4.636891
dtype: float64

In [304]:
len(users)

1000

In [288]:
mov_data.loc[mov_data.popularity<20].sort_values(by="impression",ascending=False)

,item,impression,popularity,avgRating
14,113315,474,11,4.500000
38,104636,114,15,4.233333
8,65357,110,18,4.027778
186,93008,88,12,3.750000
40,27423,85,13,4.269231
77,95175,74,10,3.850000
26,73868,74,14,3.928571
36,107410,69,15,4.200000
1,83182,68,13,3.884615
29,92954,62,17,3.882353


In [305]:
pop_mov_data.loc[pop_mov_data.popularity<20].sort_values(by="popularity",ascending=True)

,item,impression,popularity,avgRating
332,100289,2,10,3.800000
402,43801,9,10,3.700000
211,26167,12,10,3.900000
372,101079,5,10,3.900000
356,89349,4,10,3.550000
343,120815,15,10,3.500000
338,41058,4,10,3.500000
328,86947,1,10,2.450000
322,85215,1,10,2.650000
313,27350,2,10,3.350000


In [273]:
pop_mov_data

,item,impression,popularity,avgRating
0,100553,251,31,4.209677
1,527,378,50054,4.310175
2,858,487,41355,4.364732
3,113315,454,11,4.500000
4,50,449,47006,4.334372
5,26587,278,402,4.174129
6,1193,286,29932,4.248079
7,65357,98,18,4.027778
8,6016,299,12937,4.235410
9,94466,174,582,4.182990


In [271]:
movie=set()
for index, row in pop_predicts.iterrows():
    
    movie_id=row['item']
    if(movie_id in movie):
        mirow=movie_impressions.loc[movie_impressions['item'] == movie_id]
        count=mirow['impression']+1
        movie_impressions.impression.loc[movie_impressions['item'] ==  movie_id] = count
    
    else:
        newrow=pd.DataFrame([[movie_id,1]], columns=['item','impression'])
        movie.add(movie_id)    
        movie_impressions=movie_impressions.append(newrow)
            

In [272]:
pop_mov_data=pd.merge(movie_impressions,movie_data,on=['item'])

In [275]:
mov_data.to_csv("movie_recommendation_impressions_ALS25Freg01filtered.csv",sep="\t",index=None)
pop_mov_data.to_csv("movie_pop_recommendation_impressions_ALS25Freg01filtered.csv",sep="\t",index=None)

In [127]:
pred_movies=set(predicts['item'].unique())
pop_pred_movies=set(pop_predicts['item'].unique())

In [133]:
len(movie_data.loc[(movie_data['item'].isin(pop_pred_movies)) & (movie_data['popularity']<20)])

230

In [132]:
len(movie_data.loc[(movie_data['item'].isin(pred_movies)) & (movie_data['popularity']<20)])

251

In [130]:
len(movie_data.loc[(movie_data['item'].isin(pred_movies)) & (movie_data['popularity']>1450)])

149

In [131]:
len(movie_data.loc[(movie_data['item'].isin(pop_pred_movies)) & (movie_data['popularity']>1450)])

144

In [206]:
len(movie_data.loc[(movie_data['item'].isin(pred_movies)) & (movie_data['popularity']>30)])

311

In [128]:
len(movie_data.loc[(movie_data['item'].isin(pred_movies)) & (movie_data['popularity']>0)])

643

In [129]:
len(movie_data.loc[(movie_data['item'].isin(pop_pred_movies)) & (movie_data['popularity']>0)])

612

In [136]:
len(movie_data.loc[(movie_data['item'].isin(pop_pred_movies)) & (movie_data['popularity']<100)])

389

In [134]:
len(movie_data.loc[(movie_data['item'].isin(pred_movies)) & (movie_data['popularity']<100)])

415

In [135]:
len(movie_data.loc[(movie_data['item'].isin(pred_movies)) & (movie_data['popularity']>100)])

227

In [137]:
len(movie_data.loc[(movie_data['item'].isin(pop_pred_movies)) & (movie_data['popularity']>100)])

222

In [138]:
len(movie_data.loc[(movie_data['item'].isin(pred_movies)) & (movie_data['popularity']>1000)])

157

In [139]:
len(movie_data.loc[(movie_data['item'].isin(pop_pred_movies)) & (movie_data['popularity']>1000)])

152

In [140]:
len(movie_data.loc[(movie_data['item'].isin(pred_movies)) & (movie_data['popularity']>100)])

227

In [141]:
len(movie_data.loc[(movie_data['item'].isin(pop_pred_movies)) & (movie_data['popularity']>100)])

222

In [142]:
len(movie_data.loc[(movie_data['item'].isin(pop_pred_movies)) & (movie_data['popularity']>100)])

222

In [314]:
user_overlap=pd.DataFrame(columns=["user","overlap"])

In [318]:

for user in users:
    frecs=set(predicts.item.loc[predicts.user == user])
    precs=set(pop_predicts.item.loc[pop_predicts.user == user])
    overlap=len(frecs.intersection(precs))
    newrow=pd.DataFrame([[user,overlap]],columns=["user","overlap"])
    user_overlap=user_overlap.append(newrow)

In [331]:
popIds=set(movie_data.item.loc[movie_data.popularity>1450])

In [335]:
len(movie_data.item)

26744

In [341]:
user_full_recs_summary=pd.DataFrame(columns=["user","popcount","unpopcount","avgscore","avgrating"])

In [342]:
for user in users:
    
    recs=predicts.loc[predicts.user == user]
    recset=set(recs.item)
    recpop=len(recs.loc[recs.item.isin(popIds)])
    recunpop=20-recpop
    recavgscore=recs.score.mean()
    recavgrating=movie_data.avgRating.loc[movie_data.item.isin(recset)].mean()
    newrow=pd.DataFrame([[user,recpop,recunpop,recavgscore,recavgrating]],columns=["user","popcount","unpopcount","avgscore","avgrating"])
    user_full_recs_summary=user_full_recs_summary.append(newrow)
    

In [349]:
user_full_recs_summary.to_csv("ALS25Freg01recsummaryfull.csv",sep=",",header=True,index=None)

In [346]:
user_pop_recs_summary=pd.DataFrame(columns=["user","popcount","unpopcount","avgscore","avgrating"])

In [347]:
for user in users:
    
    recs=pop_predicts.loc[pop_predicts.user == user]
    recset=set(recs.item)
    recpop=len(recs.loc[recs.item.isin(popIds)])
    recunpop=20-recpop
    recavgscore=recs.score.mean()
    recavgrating=movie_data.avgRating.loc[movie_data.item.isin(recset)].mean()
    newrow=pd.DataFrame([[user,recpop,recunpop,recavgscore,recavgrating]],columns=["user","popcount","unpopcount","avgscore","avgrating"])
    user_pop_recs_summary=user_pop_recs_summary.append(newrow)
    

In [350]:
user_pop_recs_summary.to_csv("ALS25Freg01recssummarypop.csv",sep=",",header=True,index=None)

In [361]:
user_diversity=pd.DataFrame(columns=['user','diversity'])

In [365]:
for user in users:
    rec_items=list(predicts.item.loc[predicts.user == user])
    count=0
    diversity=0
    for item in rec_items:
        if rec_items.index(item) < (len(rec_items)-1): 
            iix1=algo.item_index_.get_loc(item)
            iix2=algo.item_index_.get_loc(rec_items[rec_items.index(item) + 1])
            iu=algo.item_features_[iix1]
            iv=algo.item_features_[iix2]
            dot_product=spatial.distance.cosine(iu,iv)
            diversity+=dot_product
            count+=1
    
    diversity=diversity/count
    newrow=pd.DataFrame([[user,diversity]],columns=['user','diversity'])
    user_diversity=user_diversity.append(newrow)
                  
    

In [370]:
user_diversity_pop=pd.DataFrame(columns=['user','diversity'])

In [371]:
for user in users:
    rec_items=list(pop_predicts.item.loc[pop_predicts.user == user])
    count=0
    diversity=0
    for item in rec_items:
        if rec_items.index(item) < (len(rec_items)-1): 
            iix1=algo.item_index_.get_loc(item)
            iix2=algo.item_index_.get_loc(rec_items[rec_items.index(item) + 1])
            iu=algo.item_features_[iix1]
            iv=algo.item_features_[iix2]
            dot_product=spatial.distance.cosine(iu,iv)
            diversity+=dot_product
            count+=1
    
    diversity=diversity/count
    newrow=pd.DataFrame([[user,diversity]],columns=['user','diversity'])
    user_diversity_pop=user_diversity_pop.append(newrow)
                  
    

In [379]:
count=0
diver=0
pred_movie_list=list(predicts.item.unique())
for movie in pred_movie_list:
    if pred_movie_list.index(movie) < (len(pred_movie_list)-1):
        iix1=algo.item_index_.get_loc(movie)
        iix2=algo.item_index_.get_loc(pred_movie_list[pred_movie_list.index(movie) + 1])
        iu=algo.item_features_[iix1]
        iv=algo.item_features_[iix2]
        dot_product=spatial.distance.cosine(iu,iv)
        diver+=dot_product
        count+=1
print(diver/count)
    
    

0.6110723562073133


In [380]:
count=0
diver=0
pred_movie_list=list(pop_predicts.item.unique())
for movie in pred_movie_list:
    if pred_movie_list.index(movie) < (len(pred_movie_list)-1):
        iix1=algo.item_index_.get_loc(movie)
        iix2=algo.item_index_.get_loc(pred_movie_list[pred_movie_list.index(movie) + 1])
        iu=algo.item_features_[iix1]
        iv=algo.item_features_[iix2]
        dot_product=spatial.distance.cosine(iu,iv)
        diver+=dot_product
        count+=1
print(diver/count)
    
    

0.6040505386899934


In [171]:
top_movies=movie_data.sort_values('popularity',ascending=False)
top_movies=set(top_movies.head(20).item)

In [172]:
top_movies

{1,
 32,
 50,
 110,
 150,
 260,
 296,
 318,
 356,
 457,
 480,
 527,
 589,
 592,
 593,
 780,
 1196,
 1210,
 2571,
 2858}

In [173]:
personalization_recs=pd.DataFrame(columns=['user','full','popularonly'])

In [174]:
for user in users:
    movlistf=set(predicts.loc[predicts['user']==user]['item'])
    movlistp=set(pop_predicts.loc[pop_predicts['user']==user]['item'])
    full=len(movlistf.intersection(top_movies))
    popularonly=len(movlistp.intersection(top_movies))
    newrow=pd.DataFrame([[user,full,popularonly]],columns=['user','full','popularonly'])
    personalization_recs=personalization_recs.append(newrow)
    

In [176]:
personalization_recs.to_csv('ALS25Fr01filteredfvsp.csv',sep=",",header=True,index=None)

In [372]:
user_diversity_pop.to_csv("ALS25FReg01user_diversity_pop.csv",sep=",",header=True,index=None)

In [368]:
user_diversity.to_csv("ALS25FReg01user_diversity.csv",sep=",",header=True,index=None)

In [326]:
user_overlap.to_csv("ALS25Freg01filteredoverlap.csv",sep=",",header=True,index=None)

In [54]:
predicts.to_csv('ALS_30F_recs_folding_in.tsv',sep='\t',header=True,index=None)

In [307]:
user_simscore.to_csv('user_simscore_folded_in_ALS_25reg01filtered.tsv',sep='\t',header=True,index=None)

# Comparing Recommendation Metrics

### Holding out 20% of popular and unpopular movies

In [411]:
users=test.user.unique().tolist()
train_full_pho=pd.DataFrame(columns=['user','item','rating'])
train_pop_pho=pd.DataFrame(columns=['user','item','rating'])

test_pho=pd.DataFrame(columns=['user','item','rating'])
popIds=set(movie_data.loc[movie_data.popularity > 1450].item)


In [412]:
for user in users:
    ufrating=train.loc[train.user == user]
    uprating=test.loc[test.user == user]
    homovies=set(uprating.sample(frac=0.2).item)
    testpop=uprating.loc[uprating.item.isin(homovies)]
    trainfull=ufrating.loc[~ufrating.item.isin(homovies)]
    trainpop=uprating.loc[~uprating.item.isin(homovies)]
    train_full_pho=train_full_pho.append(trainfull,sort=True)
    train_pop_pho=train_pop_pho.append(trainpop,sort=True)
    test_pho=test_pho.append(testpop,sort=True)
    

In [413]:
train_full_upo=pd.DataFrame(columns=['user','item','rating'])
test_upo=pd.DataFrame(columns=['user','item','rating'])
train_pop=pd.DataFrame(columns=['user','item','rating'])

In [414]:
unpopIds=set(movie_data.item.unique())-popIds

In [415]:
for user in users:
    ufrating=train.loc[train.user==user]
    unpopratings=ufrating.loc[ufrating.item.isin(unpopIds)]
    popratings=ufrating.loc[~ufrating.item.isin(unpopIds)]
    if(len(unpopratings)>5):
        homovies=set(unpopratings.sample(frac=0.2).item)
        trainfull=popratings.append(unpopratings.loc[~unpopratings.item.isin(homovies)],sort=True)
        train_full_upo=train_full_upo.append(trainfull,sort=True)
        test_upo=test_upo.append(unpopratings.loc[unpopratings.item.isin(homovies)],sort=True)
        train_pop=train_pop.append(test.loc[test.user == user],sort=True)
        

### Training and Generating Recommendations for our popular held out training sets

In [416]:
trainfullphoumat,trainfullphoix=algo.fold_in(train_full_pho)
trainpopphoumat,trainpopphoix=algo.fold_in(train_pop_pho)
trainfullupoumat,trainfullupoix=algo.fold_in(train_full_upo)
trainpopupoumat,trainpopupoix=algo.fold_in(train_pop)

In [417]:
train_tu=train.loc[train.user.isin(users)]

In [ ]:
recommend_for_users(algo,users,user_index,user_matrix,predicts,train)

#### generating recommendations for the popular held-out sets

In [418]:
trainfullphopredicts=pd.DataFrame(columns=['user','item','score','rank'])
trainpopphopredicts=pd.DataFrame(columns=['user','item','score','rank'])
trainfullphopredicts=recommend_for_users(algo,list(train_full_pho.user.unique()),trainfullphoix,trainfullphoumat,trainfullphopredicts,train_full_pho,20)
trainpopphopredicts=recommend_for_users(algo,list(train_pop_pho.user.unique()),trainpopphoix,trainpopphoumat,trainpopphopredicts,train_pop_pho,20)

In [419]:
trainfullphopredicts=trainfullphopredicts.reset_index(drop=True)
trainpopphopredicts=trainpopphopredicts.reset_index(drop=True)
trainfullphopredicts=trainfullphopredicts.reset_index(drop=True)
trainpopphopredicts=trainpopphopredicts.reset_index(drop=True)

In [302]:
import lenskit.metrics.topn as topn  
from lenskit.topn import RecListAnalysis 

In [313]:
user_pho_ndcg=pd.DataFrame(columns=['user','ndcg-full','ndcg-pop'])
for user in users:
    predictsf=trainfullphopredicts.loc[trainfullphopredicts.user == user]
    predictsp=trainpopphopredicts.loc[trainpopphopredicts.user == user]
    truth=test_pho.loc[test_pho.user == user]
    ndcgf=topn.ndcg(predictsf,truth)
    ndcgp=topn.ndcg(predictsp,truth)
    newrow=pd.DataFrame([[user,ndcgf,ndcgp]],columns=['user','ndcg-full','ndcg-pop'])
    user_pho_ndcg=user_pho_ndcg.append(newrow,sort=True)

In [420]:
test_pho_rel=test_pho.copy()

In [421]:
pos=test_pho.rating>0

In [422]:
test_pho_rel.loc[pos,"rating"]=1

In [423]:
neg=test_pho.rating<0
test_pho_rel.loc[neg,"rating"]=0

In [425]:
topn.ndcg(trainfullphopredicts,test_pho_rel)

0.23600815663784255

In [426]:
topn.ndcg(trainpopphopredicts,test_pho_rel)

0.23061508149821414

In [427]:
topn.precision(trainfullphopredicts,test_pho_rel)

0.1898

In [428]:
topn.precision(trainpopphopredicts,test_pho_rel)

0.18595

In [452]:
topn.recall(trainfullphopredicts,test_pho_rel)

0.19522731948158814

In [454]:
topn.recall(trainpopphopredicts,test_pho_rel)

0.1912672289652335

In [520]:
trainfullupopredicts=pd.DataFrame(columns=['user','item','score','rank'])
trainpopupopredicts=pd.DataFrame(columns=['user','item','score','rank'])
trainfullupopredicts=recommend_for_users(algo,list(train_full_upo.user.unique()),trainfullupoix,trainfullupoumat,trainfullupopredicts,train_full_upo,50)
trainpopupopredicts=recommend_for_users(algo,list(train_pop.user.unique()),trainpopupoix,trainpopupoumat,trainpopupopredicts,train_pop,50)

In [521]:
test_upo_rel=test_upo.copy()
pos=test_upo.rating>0
neg=test_upo.rating<0
test_upo_rel.loc[pos,"rating"]=1
test_upo_rel.loc[neg,"rating"]=0

In [522]:
topn.ndcg(trainfullupopredicts,test_upo_rel)

5.40348916871492

In [523]:
topn.ndcg(trainpopupopredicts,test_upo_rel)

6.222710399847571

In [524]:
topn.precision(trainfullupopredicts,test_upo_rel)

0.5113687600644122

In [525]:
topn.precision(trainpopupopredicts,test_upo_rel)

0.5958132045088567

In [526]:
topn.recall(trainfullupopredicts,test_upo_rel)

8.441254651780968

In [527]:
topn.recall(trainpopupopredicts,test_upo_rel)

9.835194045720362

In [335]:
def predictui(train,uindex,iindex,umat,imat):
    res=pd.DataFrame(columns=['user','item','rating'])
    users=list(train.user.unique())
    for index,row in train.iterrows():
        user=row['user']
        item=row['item']
        uix=uindex.get_loc(user)
        iix=iindex.get_loc(item)
        uv=umat[uix]
        iv=imat[iix]
        pdt=np.dot(uv,iv)
        newrow=pd.DataFrame([[user,item,pdt]],columns=['user','item','rating'])
        res=res.append(newrow)
        
    return res


In [336]:
from lenskit.metrics import predict

In [492]:
preds=predictui(test_pho,trainfullphoix,algo.item_index_,trainfullphoumat,algo.item_features_)

In [493]:

preds=preds.reset_index(drop=True)
diff=preds['rating']-test_pho['rating']
sqdif=diff.apply(np.square)
np.sqrt(sqdif.mean())

0.9111948007773318

In [495]:
predpop=predictui(test_pho,trainpopphoix,algo.item_index_,trainpopphoumat,algo.item_features_)

In [496]:
predpop=preds.reset_index(drop=True)
diffpop=predpop['rating']-test_pho['rating']
sqdiffpop=diffpop.apply(np.square)
np.sqrt(sqdiffpop.mean())

0.9111948007773318

In [499]:
test_upo=test_upo.reset_index(drop=True)

In [500]:
predupofull=predictui(test_upo,trainfullupoix,algo.item_index_,trainfullupoumat,algo.item_features_)

In [501]:
predupofull=predupofull.reset_index(drop=True)
dfull=predupofull['rating']-test_upo['rating']
sqdfull=dfull.apply(np.square)
np.sqrt(sqdfull.mean())

0.8782724085968678

In [502]:
predupopop=predictui(test_upo,trainpopupoix,algo.item_index_,trainpopupoumat,algo.item_features_)

In [503]:
predupopop=predupopop.reset_index(drop=True)
dfpop=predupopop['rating']-test_upo['rating']
sqdfpop=dfpop.apply(np.square)
np.sqrt(sqdfpop.mean())

0.8944323764009656

# Splitting User Data into K-Crossfolds for analyzing prediction effect on removing 

In [458]:
from lenskit import crossfold as xf

In [529]:
rmse_vals=pd.DataFrame(columns=['dataset','rmse'])
count=0
fullr=train.loc[train.user.isin(users)]

for tr, tst in xf.partition_users(fullr,5,xf.SampleFrac(0.2)):
    count+=1
    umat,uindex=algo.fold_in(tr)
    preds=predictui(tst,uindex,algo.item_index_,umat,algo.item_features_)
    preds=preds.reset_index(drop=True)
    tst=tst.reset_index(drop=True)
    diff=preds['rating']-tst['rating']
    sqdiff=diff.apply(np.square)
    rmse=np.sqrt(sqdiff.mean())
    newrow=pd.DataFrame([[count,rmse]],columns=['dataset','rmse'])
    rmse_vals=rmse_vals.append(newrow)
    

In [539]:

for tr, tst in xf.partition_users(test,5,xf.SampleFrac(0.2)):
    count+=1
    umat,uindex=algo.fold_in(tr)
    preds=predictui(tst,uindex,algo.item_index_,umat,algo.item_features_)
    preds=preds.reset_index(drop=True)
    tst=tst.reset_index(drop=True)
    diff=preds['rating']-tst['rating']
    sqdiff=diff.apply(np.square)
    rmse=np.sqrt(sqdiff.mean())
    newrow=pd.DataFrame([[count,rmse]],columns=['dataset','rmse'])
    rmse_vals=rmse_vals.append(newrow)

In [540]:
rmse_vals

,dataset,rmse
0,1,3.111918
0,2,3.010006
0,3,2.991849
0,4,3.094607
0,5,3.070414
0,6,3.070124
0,7,3.006328
0,8,3.027921
0,9,3.034778
0,10,3.064291


In [477]:
len(tst.user.unique())

200

In [537]:
np.array_equal(fullr.user.unique(),test.user.unique())

True